In [ ]:
import logging

import numpy as np
import pandas as pd

from copulas.multivariate.GaussianCopula import GaussianCopula
from copulas.utils import get_dataset, datasets_test

LOGGER = logging.getLogger(__name__)

In [ ]:
# We run the copula against all datasets available. 
results = datasets_test(GaussianCopula)

In [ ]:
results

In [ ]:
# We take some failing dataset and see the details
FILE_NAME = 'glass_1_train.csv'

In [ ]:
copula = GaussianCopula()
data = get_dataset(FILE_NAME)

copula.fit(data)
print(copula)

In [ ]:
samples = copula.sample(data.shape[0])
samples.head(5)

In [ ]:
column = 3

data_hist = data[column].hist(bins=25, figsize=(18,12), weights=np.zeros_like(data[column]) + 1. / data[column].size)
samples_hist = samples[column].hist(bins=25, figsize=(18,12), weights=np.zeros_like(samples[column]) + 1. / samples[column].size)